Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [330]:
import requests
import json
import pandas as pd
import time
import sys
import re

from bs4 import BeautifulSoup as bs
from requests.exceptions import HTTPError

In [331]:
PATH_CACHE = "../resource/cache_html.txt"
PATH_PARAMS = "../resource/params.txt"
URL_HH = "https://surgut.hh.ru"
URL_SUPERJOB = "https://www.superjob.ru"

Чтение настроек

In [332]:
def get_user_agent():
    with open(PATH_PARAMS, "r") as f:
        return json.load(f)["user-agent"]

Получение HTML кода по ссылке

In [333]:
def get_html(url):
    try:
        response = requests.get(url = url, headers = {"User-Agent": get_user_agent()})
        if response.ok == True:
            return (True, response.text)
        else:
            response.raise_for_status()
    except Exception as err:
        return (False, f'Ошибка: {err}')

Загрузка кэшированных данных из файла

In [334]:
def load_data():
    try:
        with open(PATH_CACHE, "r") as f:
            return json.load(f)
    except Exception as err:
        return {}

Чтение данных HTML из файла

In [335]:
def read_data(cache, url):
    try:
        return (True, cache[url])
    except Exception as err:
        return (False, f'Ошибка: {err}')

Запись данных HTML в файл

In [336]:
def save_data(cache, url, html):
    cache[url] = html
    with open(PATH_CACHE, "w") as f:
        return json.dump(cache, f)

Вывод статуса обработки

In [337]:
def show_status(text, sleep):
    sys.stdout.write(text)
    sys.stdout.flush()
    time.sleep(sleep)

Чтение HTML с возможностью кэширования

In [338]:
def get_html_cache(use_cache, cache, link):
    try_caching = use_cache
    sleep = 0
    
    #Если считать данные из кеша не получилось или кэширование выключено...
    if try_caching:
        res, text = read_data(cache, link)
        if not res:
            try_caching = False
    
    #...то достаём данные с сайта
    if not try_caching:
        res, text = get_html(link)
        save_data(cache, link, text)
        sleep = 1
        
    return(res, text, sleep)

Парсинг зарплаты

In [339]:
def parse_salary(salary):
    regex = re.search('^(от\s|до\s)?([\d\s]+)([-—][\d\s]+)?\s(\D+)$', salary)

    low = regex.group(2)
    if low:
        low = low.replace(" ", "").replace(u"\u00A0", "") 
    else:
        low = ""
    
    high = regex.group(3)
    if high:
        high = high.replace(" ", "").replace(u"\u00A0", "") .replace("-", "").replace("—", "")
    else:
        high = ""

    if not regex.group(1) and not high:
        high = low

    currency = regex.group(4)

    return (low, high, currency)

Парсинг hh.ru

In [340]:
def parse_hh(html):
    result = []
    parsed_html = bs(html, 'lxml')

    div_vacancies = parsed_html.find('div', {'class':'vacancy-serp'}).find_all(
        'div', {'class':'vacancy-serp-item'})

    for div_vacancy in div_vacancies:
        data = {}
        try:
            div_vacancy_name = div_vacancy.find('a', {'class':'bloko-link HH-LinkModifier'})
            vacancy_name = div_vacancy_name.getText()
            vacancy_link = div_vacancy_name['href']
        except Exception as e:
            vacancy_name = ''
            vacancy_link = ''
            
        try:
            salary_low, salary_high, currency = parse_salary(
                div_vacancy.find('div', {'class':'vacancy-serp-item__compensation'}).getText())
        except Exception as e:
            salary_low = ''
            salary_high = ''
            currency = ''

        try:
            employer = div_vacancy.find('a', {'class':'bloko-link bloko-link_secondary HH-AnonymousIndexAnalytics-Recommended-Company'}).getText()
        except Exception as e:
            employer = ''

        data['Вакансия'] = f'<a href="{vacancy_link}">{vacancy_name}</a>'
        data['Работодатель'] = employer
        data['Зарплата (мин.)'] = salary_low
        data['Зарплата (макс.)'] = salary_high
        data['Валюта'] = currency
        data['Сайт'] = f'<a href="{URL_HH}">hh.ru</a>'
        result.append(data)
    
    #Ссылка на следующую страницу
    try:
        link = URL_HH + \
            parsed_html.find('a', {'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    except Exception as e:
        link = ''

    return(link, result)

Парсинг superjob.ru

In [351]:
def parse_superjob(html):
    result = []
    link = ''
    parsed_html = bs(html, 'lxml')

    block = parsed_html.find('div', {'class':'_1ID8B'})
    if block:
        div_vacancies = block.find_all('div', {'class':'_3syPg _3P0J7 _9_FPy'})
        
        for div_vacancy in div_vacancies:
            data = {}
            try:
                vacancy_name = div_vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText()
            except Exception as e:
                vacancy_name = ''

            try:
                vacancy_link = div_vacancy.find('a', {'class': re.compile("icMQ_ _1QIBo.*_2JivQ _3dPok")})['href']
            except Exception as e:
                vacancy_link = ''

            try:
                salary_low, salary_high, currency = parse_salary(
                    div_vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText())
            except Exception as e:
                salary_low = ''
                salary_high = ''
                currency = ''

            try:
                employer = div_vacancy.find('a', {'class': re.compile("icMQ_ _205Zx.*Vm5jz")}).getText()
            except Exception as e:
                employer = ''

            data['Вакансия'] = f'<a href="{URL_SUPERJOB}{vacancy_link}">{vacancy_name}</a>'
            data['Работодатель'] = employer
            data['Зарплата (мин.)'] = salary_low
            data['Зарплата (макс.)'] = salary_high
            data['Валюта'] = currency
            data['Сайт'] = f'<a href="{URL_SUPERJOB}">superjob.ru</a>'
            result.append(data)

        #Ссылка на следующую страницу
        try:
            link = URL_SUPERJOB + \
                parsed_html.find('a', {'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'})['href']
        except Exception as e:
            link = ''

    return(link, result)

Сбор информации о вакансиях

In [374]:
vacancy_search = input('Название вакансии: ')

use_cache = True #Приоритет чтения информации из файла, а не с сайта

all_vacancies = pd.DataFrame({'Вакансия': [], 
                              'Работодатель': [],
                              'Зарплата (мин.)': [], 
                              'Зарплата (макс.)': [], 
                              'Валюта': [],
                              'Сайт': []})

if vacancy_search:
    cache = load_data()

    #hh.ru
    link = f'{URL_HH}/search/vacancy?clusters=true&enable_snippets=true&text={vacancy_search}&showClusters=false'
    page = 0

    while link:
        res, text, sleep = get_html_cache(use_cache, cache, link)

        if res:
            link, data = parse_hh(text)
            if data:
                all_vacancies = all_vacancies.append(data, ignore_index=True)
        else:
            print(text)
            
        page += 1
        show_status(f"\rОбработано страниц hh.ru: {page}", sleep)

    show_status("\n", 0)
    
    #superjob.ru
    link = f'{URL_SUPERJOB}/vacancy/search/?keywords={vacancy_search}&geo%5Bc%5D%5B0%5D=1'
    page = 0
    
    while link:
        res, text, sleep = get_html_cache(use_cache, cache, link)

        if res:
            link, data = parse_superjob(text)
            if data:
                all_vacancies = all_vacancies.append(data, ignore_index=True)
        else:
            print(text)
            
        page += 1
        show_status(f"\rОбработано страниц superjob.ru: {page}", sleep)

    show_status("\n", 0)

all_vacancies.style

Название вакансии: Python
Обработано страниц hh.ru: 100
Обработано страниц superjob.ru: 4


,Вакансия,Работодатель,Зарплата (мин.),Зарплата (макс.),Валюта,Сайт
0,Senior Python Developer,ДомКлик,,,,hh.ru
1,Программист Python,GRISSLI,90000,,руб.,hh.ru
2,Junior data engineer (внедрение моделей),Сбербанк для экспертов,,,,hh.ru
3,Web-программист Python,Сионик,120000,,руб.,hh.ru
4,Python Developer,ООО СК хайникс мемори солюшнс Восточная Европа,,,,hh.ru
5,Python Back-end Developer,ООО Эпика Девелопмент,,,,hh.ru
6,Data Analyst (Senior),Сбербанк для экспертов,,,,hh.ru
7,Ведущий разработчик Python,CATAPULTO.RU,160000,,руб.,hh.ru
8,Веб-аналитик,ООО ЮрСпектр,2100,2200,бел. руб.,hh.ru
9,DevOps engineer,"Мобильные ТелеСистемы (МТС), Беларусь",1500,,бел. руб.,hh.ru
